Here we estimate the following conditional expectation function:
$$
\mathbb{E}[valuation_i|log\_sellerfeedbackscore_i]=\alpha+\beta_1 log\_sellerfeedbackscore_i,
$$
using listings of apple tablets where the number of bids received is strictly between 3 and 12.

In [6]:
import pandas as pd
import numpy as np
from scipy import optimize
from main import estimate_median, get_loss_function, transform_covariates

data = pd.read_csv("../../data/demeaned.csv")
df = data.groupby(["id", "ispolice", "sellerfeedbackscore", "bidcount", "apple", "amazon"])["bids"].apply(lambda x: x.values).reset_index()

In [2]:
apple_bids = list(df[(df.apple == 1) & (df.ispolice == 1)].bidcount.value_counts().index)
include = df[(df.bidcount > 3) & (df.bidcount < 12) & (df.apple == 1) & (df.bidcount.isin(apple_bids))]

bids = list(include.bids)
logged_feedback = np.log(include.sellerfeedbackscore+1)
covariates = [[cov] for cov in list(logged_feedback)]

covariates = [[cov] for cov in transform_covariates([cov[0] for cov in covariates],100)]

/Users/liumukun/Desktop/projects/eme/analysis/estimation/main.py:300: RuntimeWarning: Mean of empty slice.
  covariates[order] = covariates[order].mean()
/Users/liumukun/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [3]:
expected_upper, expected_lower = estimate_mean(bids, covariates, (0,9))

calculating values for covariate: [6.467698726104353] (1/100)
total time elapsed: 0.00021927899999996114s


/Users/liumukun/anaconda3/lib/python3.8/site-packages/scipy/stats/_distn_infrastructure.py:1695: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  return integrate.quad(self._mom_integ1, 0, 1, args=(m,)+args)[0]


calculating values for covariate: [9.481255893085342] (2/100)
total time elapsed: 54.418174871s
calculating values for covariate: [5.3471075307174685] (3/100)
total time elapsed: 96.973789484s
calculating values for covariate: [4.31748811353631] (4/100)
total time elapsed: 143.44676848600002s
calculating values for covariate: [6.236369590203704] (5/100)
total time elapsed: 202.37861991800003s
calculating values for covariate: [6.329720905522696] (6/100)
total time elapsed: 249.94151110800001s
calculating values for covariate: [3.5553480614894135] (7/100)
total time elapsed: 295.35027310699996s
calculating values for covariate: [1.3862943611198906] (8/100)
total time elapsed: 340.07867600599997s
calculating values for covariate: [4.30406509320417] (9/100)
total time elapsed: 383.186163015s
calculating values for covariate: [3.295836866004329] (10/100)
total time elapsed: 435.89026462199996s
calculating values for covariate: [7.313015299394591] (11/100)
total time elapsed: 491.456383464s

In [7]:
def loss_function(c):
    a, b = c
    cef = lambda cov: a+b*cov[0]
    return get_loss_function(covariates, expected_upper, expected_lower, cef)

b_hat = optimize.brute(loss_function, ranges=[(0,2), (-1,1)])
# interval_lower = optimize.newton(lambda a_l, b_l: loss_function(a_l, b_l)-loss_function(b_hat)-10, b_hat-0.1)
# interval_upper = optimize.newton(lambda b_l, b_u: loss_function(b_l, b_u)-loss_function(b_hat)-10, b_hat+0.1)

In [8]:
b_hat

array([0.70420163, 0.00736892])